# 1. Импортирование библиотек и парсинг данных

In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import re
import os
from glob import glob
from torch.utils.data import TensorDataset, DataLoader
from torchvision import models, transforms
from tqdm import tqdm
# Отключаем проверку
import warnings
warnings.filterwarnings("ignore")

In [179]:
list_bmp = [] # Массив путей к bmp
list_frame = [] # Массив путей к frame
def parsing_bmp(dir):
    list = os.listdir(dir)    
    for item in glob(dir+'\\'+'*.bmp'):
        list_bmp.append(item)
    for item in glob(dir+'\\'+'*.frame'):
        list_frame.append(item)
    for dir_for in list:
        if os.path.isdir(dir+'\\'+dir_for):
            parsing_bmp(dir+'\\'+dir_for)
parsing_bmp('.')
list_bmp[:10]

['.\\DATAFRAME\\FRAMES\\frame0000.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0000.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0001.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0002.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0003.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0004.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0005.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0006.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0007.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0008.bmp']

In [180]:
list_frame[:10]

['.\\DATAFRAME\\FRAMES\\frame0000.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0000.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0001.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0002.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0003.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0004.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0005.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0006.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0007.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0008.frame']

In [181]:
print(f'Файлов формата bmp {len(list_bmp)} и формата frame {len(list_frame)}')

Файлов формата bmp 7808 и формата frame 7808


# 2. Чтение изображения

In [192]:
images = []
for file in tqdm(list_bmp):
    try:
        image = cv2.imread(file, cv2.IMREAD_COLOR) 
        image = cv2.resize(image, dsize=(960, 600))
        images.append(image)
    except:
        pass
images

100%|█████████████████████████████████████████████████████████████████████████████| 7808/7808 [00:29<00:00, 263.30it/s]


[array([[[ 99,  99,  99],
         [ 99,  99,  99],
         [ 97,  97,  97],
         ...,
         [129, 129, 129],
         [124, 124, 124],
         [122, 122, 122]],
 
        [[ 97,  97,  97],
         [ 97,  97,  97],
         [ 98,  98,  98],
         ...,
         [128, 128, 128],
         [127, 127, 127],
         [127, 127, 127]],
 
        [[ 99,  99,  99],
         [ 99,  99,  99],
         [ 97,  97,  97],
         ...,
         [130, 130, 130],
         [131, 131, 131],
         [130, 130, 130]],
 
        ...,
 
        [[ 94,  94,  94],
         [ 94,  94,  94],
         [ 93,  93,  93],
         ...,
         [ 93,  93,  93],
         [ 91,  91,  91],
         [ 95,  95,  95]],
 
        [[ 93,  93,  93],
         [ 92,  92,  92],
         [ 90,  90,  90],
         ...,
         [ 94,  94,  94],
         [ 94,  94,  94],
         [ 95,  95,  95]],
 
        [[ 90,  90,  90],
         [ 88,  88,  88],
         [ 85,  85,  85],
         ...,
         [100, 100, 100],
  

In [193]:
len(image)

600